In [1]:
def findPokerHand(hand):
    ranks = []
    suits = []
    possibleRanks = []
 
    for card in hand:
        if len(card) == 2:
            rank = card[0]
            suit = card[1]
        else:
            rank = card[0:2]
            suit = card[2]
        if rank == "A":
            rank = 14
        elif rank == "K":
            rank = 13
        elif rank == "Q":
            rank = 12
        elif rank == "J":
            rank = 11
        ranks.append(int(rank))
        suits.append(suit)
 
    sortedRanks = sorted(ranks)
 
    # Royal Flush and Straight Flush and Flush
    if suits.count(suits[0]) == 5: # Check for Flush
        if 14 in sortedRanks and 13 in sortedRanks and 12 in sortedRanks and 11 in sortedRanks \
                and 10 in sortedRanks:
            possibleRanks.append(10)
        elif all(sortedRanks[i] == sortedRanks[i - 1] + 1 for i in range(1, len(sortedRanks))):
            possibleRanks.append(9)
        else:
            possibleRanks.append(6) # -- Flush
 
    # Straight
    # 10 11 12 13 14
    #  11 == 10 + 1
    if all(sortedRanks[i] == sortedRanks[i - 1] + 1 for i in range(1, len(sortedRanks))):
        possibleRanks.append(5)
 
    handUniqueVals = list(set(sortedRanks))
 
    # Four of a kind and Full House
    # 3 3 3 3 5   -- set --- 3 5 --- unique values = 2 --- Four of a kind
    # 3 3 3 5 5   -- set -- 3 5 ---- unique values = 2 --- Full house
    if len(handUniqueVals) == 2:
        for val in handUniqueVals:
            if sortedRanks.count(val) == 4:  # --- Four of a kind
                possibleRanks.append(8)
            if sortedRanks.count(val) == 3:  # --- Full house
                possibleRanks.append(7)
 
    # Three of a Kind and Pair
    # 5 5 5 6 7 -- set -- 5 6 7 --- unique values = 3   -- three of a kind
    # 8 8 7 7 2 -- set -- 8 7 2 --- unique values = 3   -- two pair
    if len(handUniqueVals) == 3:
        for val in handUniqueVals:
            if sortedRanks.count(val) == 3:  # -- three of a kind
                possibleRanks.append(4)
            if sortedRanks.count(val) == 2:  # -- two pair
                possibleRanks.append(3)
 
    # Pair
    # 5 5 3 6 7 -- set -- 5 3 6 7 - unique values = 4 -- Pair
    if len(handUniqueVals) == 4:
        possibleRanks.append(2)
 
    if not possibleRanks:
        possibleRanks.append(1)
    # print(possibleRanks)
    pokerHandRanks = {10: "Royal Flush", 9: "Straight Flush", 8: "Four of a Kind", 7: "Full House", 6: "Flush",
                      5: "Straight", 4: "Three of a Kind", 3: "Two Pair", 2: "Pair", 1: "High Card"}
    output = pokerHandRanks[max(possibleRanks)]
    print(hand, output)
    return output
 
 
if __name__ == "__main__":
    findPokerHand(["KH", "AH", "QH", "JH", "10H"])  # Royal Flush
    findPokerHand(["QC", "JC", "10C", "9C", "8C"])  # Straight Flush
    findPokerHand(["5C", "5S", "5H", "5D", "QH"])  # Four of a Kind
    findPokerHand(["2H", "2D", "2S", "10H", "10C"])  # Full House
    findPokerHand(["2D", "KD", "7D", "6D", "5D"])  # Flush
    findPokerHand(["JC", "10H", "9C", "8C", "7D"])  # Straight
    findPokerHand(["10H", "10C", "10D", "2D", "5S"])  # Three of a Kind
    findPokerHand(["KD", "KH", "5C", "5S", "6D"])  # Two Pair
    findPokerHand(["2D", "2S", "9C", "KD", "10C"])  # Pair
    findPokerHand(["KD", "5H", "2D", "10C", "JH"])  # High Card

['KH', 'AH', 'QH', 'JH', '10H'] Royal Flush
['QC', 'JC', '10C', '9C', '8C'] Straight Flush
['5C', '5S', '5H', '5D', 'QH'] Four of a Kind
['2H', '2D', '2S', '10H', '10C'] Full House
['2D', 'KD', '7D', '6D', '5D'] Flush
['JC', '10H', '9C', '8C', '7D'] Straight
['10H', '10C', '10D', '2D', '5S'] Three of a Kind
['KD', 'KH', '5C', '5S', '6D'] Two Pair
['2D', '2S', '9C', 'KD', '10C'] Pair
['KD', '5H', '2D', '10C', 'JH'] High Card


In [1]:
from ultralytics import YOLO
import cv2
import cvzone
import math
 
cap = cv2.VideoCapture(0)  # For Webcam
cap.set(3, 1280)
cap.set(4, 720)
 
model = YOLO("playingCards.pt")
classNames = ['10C', '10D', '10H', '10S',
              '2C', '2D', '2H', '2S',
              '3C', '3D', '3H', '3S',
              '4C', '4D', '4H', '4S',
              '5C', '5D', '5H', '5S',
              '6C', '6D', '6H', '6S',
              '7C', '7D', '7H', '7S',
              '8C', '8D', '8H', '8S',
              '9C', '9D', '9H', '9S',
              'AC', 'AD', 'AH', 'AS',
              'JC', 'JD', 'JH', 'JS',
              'KC', 'KD', 'KH', 'KS',
              'QC', 'QD', 'QH', 'QS']
 
while True:
    success, img = cap.read()
    results = model(img, stream=True)
    hand = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
 
            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)
 
            if conf > 0.5:
                hand.append(classNames[cls])
 
    print(hand)
    hand = list(set(hand))
    print(hand)
    if len(hand) == 5:
        results =findPokerHand(hand)
        print(results)
        cvzone.putTextRect(img, f'Your Hand: {results}', (300, 75), scale=3, thickness=5)
    cv2.imshow("Image", img)
    cv2.waitKey(1)


0: 480x640 (no detections), 1902.0ms
Speed: 17.5ms preprocess, 1902.0ms inference, 11.6ms postprocess per image at shape (1, 3, 480, 640)
[]
[]

0: 480x640 1 BACK, 2 KHs, 1937.0ms
Speed: 25.2ms preprocess, 1937.0ms inference, 11.1ms postprocess per image at shape (1, 3, 480, 640)
['QC', 'QC']
['QC']

0: 480x640 1 BACK, 2 KHs, 2150.5ms
Speed: 3.3ms preprocess, 2150.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
['QC', 'QC']
['QC']

0: 480x640 2 KHs, 1537.0ms
Speed: 6.3ms preprocess, 1537.0ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)
['QC', 'QC']
['QC']

0: 480x640 (no detections), 1498.0ms
Speed: 3.6ms preprocess, 1498.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
[]
[]

0: 480x640 (no detections), 1493.2ms
Speed: 0.0ms preprocess, 1493.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
[]
[]

0: 480x640 2 KHs, 1513.4ms
Speed: 0.0ms preprocess, 1513.4ms inference, 2.5ms postprocess per image at shape 

['AC', '2H', '6C', 'AC']
['AC', '2H', '6C']

0: 480x640 1 2H, 1 6C, 2 ACs, 1514.8ms
Speed: 6.3ms preprocess, 1514.8ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
['AC', '2H', '6C', 'AC']
['AC', '2H', '6C']

0: 480x640 1 2H, 1 6C, 2 ACs, 1474.6ms
Speed: 5.0ms preprocess, 1474.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
['AC', '2H', '6C']
['AC', '2H', '6C']

0: 480x640 1 2H, 1 2S, 1 6C, 1493.7ms
Speed: 5.7ms preprocess, 1493.7ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)
['2S', '6C', '2H']
['2H', '6C', '2S']

0: 480x640 (no detections), 1508.1ms
Speed: 4.3ms preprocess, 1508.1ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)
[]
[]

0: 480x640 1 2H, 1 2S, 1 6C, 1 AC, 1504.0ms
Speed: 3.0ms preprocess, 1504.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
['AC', '2H', '2S', '6C']
['AC', '2H', '2S', '6C']

0: 480x640 1 2H, 1 2S, 1 6C, 1 AC, 1505.2ms
Speed: 2.4ms preprocess, 1505.2ms infe

NameError: name 'findPokerHand' is not defined